In [1]:
# import the necessary packages
from imutils import face_utils
import dlib
import cv2

In [2]:
import numpy as np

In [3]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
p = "shape_predictor_194_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

#### 特征七：嘴唇颜色变化

In [4]:
#假设一个视频中人数恒定且嘴唇可以对应上,最终返回特征为lipcolorchg
#三通道版本
cap = cv2.VideoCapture('../data/aktnlyqpah.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#上下嘴唇对应点索引
tlipindex = [i for i in range(152,166)]+[i for i in range(180,194)]
blipindex = [i for i in range(11,21)]+[i for i in range(22,26)]+[i for i in range(166,180)]
vlip = []
for _ in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)

    lipavgs = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        mask = np.zeros((image.shape[0], image.shape[1]))
        cv2.fillPoly(mask, [shape[tlipindex], shape[blipindex]], 1)
        pixelcnt = mask.sum()
        mask = mask.astype(np.bool)
        out = np.zeros_like(image)
        out[mask] = image[mask]
        #计算嘴唇色度均值，rgb通道分开计算
        lipsum = [out[:,:,i].sum() for i in range(3)]
        lipavg = lipsum/pixelcnt
        lipavg = lipavg.tolist()
        lipavgs.append(lipavg)
    vlip.append(lipavgs)

#vlip为三维数组：[帧数，嘴唇数，rgb通道]
# print(vlip)
vlip = np.array(vlip)
number = len(rects)
lipcolorchg = []
for m in range(number):
    for n in range(3):
        lipcolorchg.append(vlip[:,m,n].std())
# print(lipcolorchg)    
#计算所有嘴唇（一般只有一个嘴唇）的所有通道波动的均值
lipcolorchg = np.array(lipcolorchg).mean()
print(lipcolorchg) 

cap.release()


6.277950837942434


In [7]:
#灰度版本,返回特征为lipcolorchg
cap = cv2.VideoCapture('../data/aktnlyqpah.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#上下嘴唇对应点索引
tlipindex = [i for i in range(152,166)]+[i for i in range(180,194)]
blipindex = [i for i in range(11,21)]+[i for i in range(22,26)]+[i for i in range(166,180)]
vlip = []
for _ in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)

    lipavgs = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        mask = np.zeros((image.shape[0], image.shape[1]))
        cv2.fillPoly(mask, [shape[tlipindex], shape[blipindex]], 1)
        pixelcnt = mask.sum()
        mask = mask.astype(np.bool)
        out = np.zeros_like(gray)
        out[mask] = gray[mask]
        #计算嘴唇色度均值
        lipsum = out.sum()
        lipavg = lipsum/pixelcnt
        lipavgs.append(lipavg)
    vlip.append(lipavgs)

#vlip为二维数组：[帧数，嘴唇数]

vlip = np.array(vlip)
number = len(rects)
lipcolorchg = []
for m in range(number):
    lipcolorchg.append(vlip[:,m].std())
    
lipcolorchg = np.array(lipcolorchg).mean()
print(lipcolorchg)

cap.release()


6.1250843355574025


#### 特征八：面部特征相对移动距离

In [9]:
#假设一个视频中人数恒定且相应人脸可以对应上,最终返回特征为fluctuation
cap = cv2.VideoCapture('../data/aktnlyqpah.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#上下嘴唇对应点索引
tlipindex = [i for i in range(152,166)]+[i for i in range(180,194)]
blipindex = [i for i in range(11,21)]+[i for i in range(22,26)]+[i for i in range(166,180)]
#眉毛对应点索引
lbrowindex = [i for i in range(92,98)]+[i for i in range(99,109)]+[i for i in range(110,114)]
rbrowindex = [i for i in range(70,76)]+[i for i in range(77,87)]+[i for i in range(88,92)]
#眼睛对应点索引
leyeindex = [i for i in range(48,54)]+[i for i in range(55,65)]+[i for i in range(66,70)]
reyeindex = [i for i in range(26,32)]+[i for i in range(33,43)]+[i for i in range(44,48)]
#鼻子对应点索引
noseindex = [i for i in range(135,152)]
    
for f in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)


    # loop over the face detections
    centers = []
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        #计算五官中心坐标
        center = [shape[lbrowindex].mean(axis=0).tolist(),
                  shape[rbrowindex].mean(axis=0).tolist(),
                  shape[leyeindex].mean(axis=0).tolist(),
                  shape[reyeindex].mean(axis=0).tolist(),
                  shape[noseindex].mean(axis=0).tolist(),
                  np.concatenate((shape[tlipindex],shape[blipindex])).mean(axis=0).tolist()]
        
        centers.append(center)
    if f == 0:
        base = centers.copy()
        fluctuation = 0
    else:
        for i in range(len(rects)):
            fluc = np.linalg.norm(np.array(centers[i])-np.array(base[i]),axis=1).std()
            if fluc > fluctuation:
                fluctuation = fluc
#                 print(fluctuation)
        base = centers.copy()
print(fluctuation)
cap.release()

3.3974655197200074


#### 特征九：眉毛间距

In [10]:
#假设一个视频中人数恒定且相应人脸可以对应上,最终返回特征为browdis
cap = cv2.VideoCapture('../data/aktnlyqpah.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

Dis = []
for f in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)


    # loop over the face detections
    dis = []
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        dis.append(np.linalg.norm(shape[70]-shape[82]))
        
    Dis.append(dis)
#Dis为二维数组[帧数，脸数]
Dis = np.array(Dis)
browdis = 0
for i in range(len(rects)):
    browdis += Dis[:,i].std()

browdis = browdis/len(rects)
print(browdis)
cap.release()

3.6508332669427084


#### 特征十：眉毛中点到鼻子距离

In [11]:
#假设一个视频中人数恒定且相应人脸可以对应上,最终返回特征为brow2nose
cap = cv2.VideoCapture('../data/aktnlyqpah.mp4')
v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

brow2noseDis = []
for f in range(v_len):
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 0)


    # loop over the face detections
    dis = []
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        dis.append(np.linalg.norm(np.concatenate((shape[70],shape[82])).mean(axis=0)-shape[143]))
        
    brow2noseDis.append(dis)
#Dis为二维数组[帧数，脸数]
brow2noseDis = np.array(brow2noseDis)
brow2nose = 0
for i in range(len(rects)):
    brow2nose += brow2noseDis[:,i].std()
#求几张脸该特征的均值
brow2nose = brow2nose/len(rects)
print(brow2nose)

cap.release()

11.694296661599912
